In [2]:
!pip3 install tensorflow-gpu
!pip3 install keras-nlp
!pip3 install pandas
!pip3 install clean-text
!pip3 install tqdm


DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
import keras_nlp
import tensorflow as tf
from tensorflow import keras

import pandas as pd
from cleantext import clean
import random

from tqdm import tqdm
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()


Using TensorFlow backend


Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.
C:\Users\Deepblue\AppData\Local\Temp\ipykernel_28648\3769467074.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


In [4]:
BATCH_SIZE = 64
EPOCHS = 40
MAX_SEQUENCE_LENGTH = 150
TEXT_VOCAB_SIZE = 7700
SUMM_VOCAB_SIZE = 3800

EMBED_DIM = 256
INTERMEDIATE_DIM = 2048
NUM_HEADS = 8


In [5]:
train_df = pd.read_json("dataset/summarization/train.json")
train_df.columns = ["id", "summary", "text"]
train_df.head()


,id,summary,text
0,13818513,Amanda baked cookies and will bring Jerry some...,Amanda: I baked cookies. Do you want some?\r\...
1,13728867,Olivia and Olivier are voting for liberals in ...,Olivia: Who are you voting for in this electio...
2,13681000,Kim may try the pomodoro technique recommended...,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa..."
3,13730747,Edward thinks he is in love with Bella. Rachel...,"Edward: Rachel, I think I'm in ove with Bella...."
4,13728094,"Sam is confused, because he overheard Rick com...",Sam: hey overheard rick say something\r\nSam:...


In [6]:
print("Text:\n",train_df["text"][0])
print("-----------------------------")
print("Summary: ", train_df["summary"][0])


Text:
 Amanda: I baked  cookies. Do you want some?
Jerry: Sure!
Amanda: I'll bring you tomorrow :-)
-----------------------------
Summary:  Amanda baked cookies and will bring Jerry some tomorrow.


In [7]:
def clean_samples(text):
    return clean(text=text, normalize_whitespace=True, no_urls=True, no_emoji=True)


In [8]:
text_pairs = []

for i,j in zip(train_df.text, train_df.summary):
    if len(i.split(" ")) < MAX_SEQUENCE_LENGTH:
        text_pairs.append((clean_samples(i),clean_samples(j)))
        

In [9]:
def train_word_piece(text_samples, vocab_size, reserved_tokens):
    word_piece_ds = tf.data.Dataset.from_tensor_slices(text_samples)
    vocab = keras_nlp.tokenizers.compute_word_piece_vocabulary(
    word_piece_ds.batch(1000).prefetch(2),
    vocabulary_size=vocab_size,
    reserved_tokens=reserved_tokens,
    )
    return vocab


In [10]:
reserved_tokens = ["[PAD]", "[UNK]", "[START]", "[END]"]

text_samples = [text_pair[0] for text_pair in text_pairs]
text_vocab = train_word_piece(text_samples, TEXT_VOCAB_SIZE, reserved_tokens)

summ_samples = [text_pair[1] for text_pair in text_pairs]
summ_vocab = train_word_piece(text_samples, SUMM_VOCAB_SIZE, reserved_tokens)

In [11]:
print("Text Tokens: ", text_vocab[100:110])
print("Summarization Tokens: ", summ_vocab[100:110])


Text Tokens:  ['don', 'with', 'this', '##s', 'was', 'there', 'like', 'your', 'about', 'he']
Summarization Tokens:  ['just', 'don', 'with', 'this', 'was', 'there', 'like', 'your', 'about', 'he']


In [12]:
len(text_vocab), len(summ_vocab)


(6453, 3612)

In [13]:
text_tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
vocabulary=text_vocab, lowercase=False
)
summ_tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
vocabulary=summ_vocab, lowercase=False
)


In [14]:
text_input_ex = text_pairs[0][0]
text_tokens_ex = text_tokenizer.tokenize(text_input_ex)
print("Text sentence: ",text_input_ex)
print("Tokens: ", text_tokens_ex)
print(
"Recovered text after detokenizing: ",
text_tokenizer.detokenize(text_tokens_ex),
)

print()

summ_input_ex = text_pairs[0][1]
summ_tokens_ex = summ_tokenizer.tokenize(summ_input_ex)
print("Summarization sentence: ", summ_input_ex)
print("Tokens: ", summ_tokens_ex)
print(
"Recovered text after detokenizing: ",
summ_tokenizer.detokenize(summ_tokens_ex),
)


Text sentence:  amanda: i baked cookies. do you want some?
jerry: sure!
amanda: i'll bring you tomorrow :-)
Tokens:  tf.Tensor(
[ 410   29   49 3453 1746   17   90   70  144  123   34  572   29  115
    4  410   29   49   10   95  303   70  172   29   16   12], shape=(26,), dtype=int32)
Recovered text after detokenizing:  tf.Tensor(b"amanda : i baked cookies . do you want some ? jerry : sure ! amanda : i ' ll bring you tomorrow : - )", shape=(), dtype=string)

Summarization sentence:  amanda baked cookies and will bring jerry some tomorrow.
Tokens:  tf.Tensor([ 421 2724  164 1898   74   99  316  593  123  177   17], shape=(11,), dtype=int32)
Recovered text after detokenizing:  tf.Tensor(b'amanda baked cookies and will bring jerry some tomorrow .', shape=(), dtype=string)


In [17]:
def preprocess_batch(text, summ):
    batch_size = tf.shape(summ)[0]
    
    text = text_tokenizer(text)
    summ = summ_tokenizer(summ)
    
    # Pad `text` to `MAX_SEQUENCE_LENGTH`.
    text_start_end_packer = keras_nlp.layers.StartEndPacker(
    sequence_length=MAX_SEQUENCE_LENGTH,
    pad_value=text_tokenizer.token_to_id("[PAD]"),
    )
    text = text_start_end_packer(text)
    
    # Add special tokens (`"[START]"` and `"[END]"`) to `summ` and pad it as well.
    summ_start_end_packer = keras_nlp.layers.StartEndPacker(
    sequence_length=MAX_SEQUENCE_LENGTH + 1,
    start_value=summ_tokenizer.token_to_id("[START]"),
    end_value=summ_tokenizer.token_to_id("[END]"),
    pad_value=summ_tokenizer.token_to_id("[PAD]"),
    )
    summ = summ_start_end_packer(summ)
    
    return (
    {
        "encoder_inputs": text,
        "decoder_inputs": summ[:, :-1],
    },
        summ[:, 1:],
    )

In [18]:
def make_dataset(pairs):
    source_texts, target_texts = zip(*pairs)
    source_texts = list(source_texts)
    target_texts = list(target_texts)
    dataset = tf.data.Dataset.from_tensor_slices((source_texts, target_texts))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(preprocess_batch, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset.shuffle(2048).prefetch(16).cache()
train_ds = make_dataset(text_pairs)

In [20]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (64, 150)
inputs["decoder_inputs"].shape: (64, 150)
targets.shape: (64, 150)


In [21]:
# Encoder
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")

x = keras_nlp.layers.TokenAndPositionEmbedding(
vocabulary_size=TEXT_VOCAB_SIZE,
sequence_length=MAX_SEQUENCE_LENGTH,
embedding_dim=EMBED_DIM,
mask_zero=True)(encoder_inputs)

encoder_outputs = keras_nlp.layers.TransformerEncoder(
intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS)(inputs=x)
encoder = keras.Model(encoder_inputs, encoder_outputs)


In [22]:
# Decoder
decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, EMBED_DIM), name="decoder_state_inputs")

x = keras_nlp.layers.TokenAndPositionEmbedding(
vocabulary_size=SUMM_VOCAB_SIZE,
sequence_length=MAX_SEQUENCE_LENGTH,
embedding_dim=EMBED_DIM,
mask_zero=True,)(decoder_inputs)


In [23]:
# Decoder
decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, EMBED_DIM), name="decoder_state_inputs")

x = keras_nlp.layers.TokenAndPositionEmbedding(
vocabulary_size=SUMM_VOCAB_SIZE,
sequence_length=MAX_SEQUENCE_LENGTH,
embedding_dim=EMBED_DIM,
mask_zero=True,)(decoder_inputs)

x = keras_nlp.layers.TransformerDecoder(
intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(decoder_sequence=x, encoder_sequence=encoded_seq_inputs)

x = keras.layers.Dropout(0.5)(x)
decoder_outputs = keras.layers.Dense(SUMM_VOCAB_SIZE, activation="softmax")(x)
decoder = keras.Model(
[
    decoder_inputs,
    encoded_seq_inputs,
],
    decoder_outputs,
)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])

transformer = keras.Model(
[encoder_inputs, decoder_inputs],
decoder_outputs,
name="transformer",)

In [24]:
transformer.summary()


Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 token_and_position_embedding (  (None, None, 256)   2009600     ['encoder_inputs[0][0]']         
 TokenAndPositionEmbedding)                                                                       
                                                                                                  
 decoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 transformer_encoder (Transform  (None, None, 256)   1315072     ['token_and_position_em

In [25]:
transformer.compile(
"adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.fit(train_ds, epochs=EPOCHS)

Epoch 1/40
197/197 [==============================] - 20s 80ms/step - loss: 1.0061 - accuracy: 0.1914
Epoch 2/40
197/197 [==============================] - 16s 80ms/step - loss: 0.8428 - accuracy: 0.2462
Epoch 3/40
197/197 [==============================] - 16s 80ms/step - loss: 0.7894 - accuracy: 0.2674
Epoch 4/40
197/197 [==============================] - 16s 81ms/step - loss: 0.7464 - accuracy: 0.2885
Epoch 5/40
197/197 [==============================] - 15s 79ms/step - loss: 0.7019 - accuracy: 0.3143
Epoch 6/40
197/197 [==============================] - 15s 77ms/step - loss: 0.6547 - accuracy: 0.3452
Epoch 7/40
197/197 [==============================] - 15s 77ms/step - loss: 0.6072 - accuracy: 0.3760
Epoch 8/40
197/197 [==============================] - 15s 77ms/step - loss: 0.5626 - accuracy: 0.4065
Epoch 9/40
197/197 [==============================] - 15s 77ms/step - loss: 0.5205 - accuracy: 0.4363
Epoch 10/40
197/197 [==============================] - 15s 77ms/step - loss: 0.482

In [29]:
def decode_sequences(input_sentences):
    batch_size = tf.shape(input_sentences)[0]
    
    # Tokenize the encoder input.
    encoder_input_tokens = text_tokenizer(input_sentences).to_tensor(
    shape=(None, MAX_SEQUENCE_LENGTH)
    )
    
    # Define a function that outputs the next token's probability given the
    # input sequence.
    def next(prompt, cache, index):
        logits = transformer([encoder_input_tokens, prompt])[:, index - 1, :]
        # Ignore hidden states for now; only needed for contrastive search.
        hidden_states = None
        return logits, hidden_states, cache
    
    # Build a prompt of length MAX_SEQUENCE_LENGTH with a start token and padding tokens.
    length = MAX_SEQUENCE_LENGTH
    start = tf.fill((batch_size, 1), summ_tokenizer.token_to_id("[START]"))
    pad = tf.fill((batch_size, length - 1), summ_tokenizer.token_to_id("[PAD]"))
    prompt = tf.concat((start, pad), axis=-1)
    
    generated_tokens = keras_nlp.samplers.GreedySampler()(
    next,
    prompt,
    end_token_id=summ_tokenizer.token_to_id("[END]"),
    index=1,  # Start sampling after start token.
    )
    generated_sentences = summ_tokenizer.detokenize(generated_tokens)
    return generated_sentences


In [31]:
test_texts = [pair[0] for pair in text_pairs]
original = [pair[1] for pair in text_pairs]
for i in range(2):
    choice = random.randint(0, len(test_texts))
    input_sentence = test_texts[choice]
    summarized = decode_sequences(tf.constant([input_sentence]))
    summarized = summarized.numpy()[0].decode("utf-8")
    summarized = (
    summarized.replace("[PAD]", "")
    .replace("[START]", "")
    .replace("[END]", "")
    .strip()
    )
    print(f"** Example {i} **")
    print("Text:", input_sentence)
    print("Prediction:", summarized)
    print("Ground Truth:", original[choice])
    print()

** Example 0 **
Text: susan: are you as bored as i am?
tom: i'm sleeping with eyes open so the prof won't realise
linda: <photo_file>
tom: nice drawings
Prediction: susan , tom and linda are bored at the university lecture .
Ground Truth: susan, tom and linda are bored at the university lecture. linda sends photo of her drawings.

** Example 1 **
Text: jeff: bro its been a while!
jeff: whats up
steve: hey what's up
steve: ya idk what happened!
steve: you haven't been around!
jeff: ya well i was with my gf
jeff: but we broke up
steve: ah that's why bro
steve: sad to hear that you broke up with your gf tho
jeff: it was rough but it was the past
jeff: wanna meet up after work?
steve: no problem
steve: at o'sheas?
jeff: np!
Prediction: jeff wasn ' t available much as he had a girfriend , but jeff decides to meet there after work .
Ground Truth: jeff wasn't available much as he had a girfriend, but they split up. jeff and gary will meet at o'sheas after work.

